## Development Activity Tracker - Maintenance
#### Important: Please run the [MNCD installer](https://www.arcgis.com/home/item.html?id=46c7512604654601ab4338f9299c5414) first if the Manage Notebook Code Dependencies (MNCD) tool has not been applied to the Notebook environment.
#### Important: Please ensure the item IDs and the source URLs are still valid.

In [ ]:
# Specify the item IDs and the source URL of the development application dataset
dev_table_itemid = "" # Hosted table layer
dev_itemid = "" # Hosted feature layer
dev_source = "https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/0aa7e480-9b48-4919-98e0-6af7615b7809/resource/060be258-88ef-4fa3-a159-cbb60c9c1d47/download/Development%20Applications%20Data.csv"
# Specify the item ID and the source URL of the regional municipal boundary
city_itemid = ""
city_source = "https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/841fb820-46d0-46ac-8dcb-d20f27e57bcc/resource/41bf97f0-da1a-46a9-ac25-5ce0078d6760/download/toronto-boundary-wgs84.zip"

### 1. Create Connection to GIS Environment

In [ ]:
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.GIS
from arcgis.gis import GIS
gis = GIS("home")

### 2. Automate Updates Using OverwriteFS Tool

In [ ]:
# Align the current working directory for notebook and scheduled tasks
import os
os.chdir("/arcgis")

In [ ]:
# Create work folder if it does not exist
work_path = "/arcgis/home/work"
if not os.path.exists(work_path):
    os.mkdir(work_path)

In [ ]:
# Import MNCD tool and OverwriteFS tool
from home.mncd import mncd
mncd.manageDependents("d45f80eb53c748e7aa3d938a46b48836")
import OverwriteFS

In [ ]:
# Update feature services
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.get
# Reference: https://www.arcgis.com/home/item.html?id=cd5819375aca40a3a7f8b3a269404c2c
dev_table = gis.content.get(dev_table_itemid)
city_item = gis.content.get(city_itemid)
dev_update = False
for data in [(dev_table, dev_source), (city_item, city_source)]:
    overwrite = OverwriteFS.overwriteFeatureService(data[0], data[1], outPath = work_path, verbose = True)
    if overwrite["success"] == True:
        print("Service Overwrite was a Success!")
        if data[0] == dev_table:
            dev_update = True
    elif overwrite["success"] == False:
        print("Service Overwrite Failed!")
        for step in overwrite["items"][-3:]:
            print(" - Action: '{}', Result: '{}'".format(step["action"], step["result"]))

### 3. Sanitize Problematic Data

#### Invalid X and Y Coordinates

In [ ]:
# Obtain development applications with invalid X and Y coordinates
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.query
if dev_update == True:
    dev_tlayer = dev_table.tables[0]
    dev_invalid_xy = dev_tlayer.query(where = "X < 290000 OR X > 340000 OR Y < 4825000 OR Y > 4860000")

In [ ]:
# Set invalid X and Y coordinates to null
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.edit_features
if dev_update == True:
    for feature in dev_invalid_xy:
        try:
            feature.attributes['X'] = None
            feature.attributes['Y'] = None
            dev_tlayer.edit_features(updates = dev_invalid_xy)
        except Exception as e:
            print(e)

#### Invalid Dates

In [ ]:
# Obtain development applications with invalid dates
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.query
if dev_update == True:
    dev_invalid_date_submitted = dev_tlayer.query(where = "DATE_SUBMITTED < '2000-01-01' OR DATE_SUBMITTED > '2200-01-01'")
    dev_invalid_hearing_date = dev_tlayer.query(where = "HEARING_DATE < '2000-01-01' OR HEARING_DATE > '2200-01-01'")

In [ ]:
# Set invalid dates to null
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.edit_features
if dev_update == True:
    for feature in dev_invalid_date_submitted:
        try:
            feature.attributes['DATE_SUBMITTED'] = None
            dev_tlayer.edit_features(updates = dev_invalid_date_submitted)
        except Exception as e:
            print(e)
    for feature in dev_invalid_hearing_date:
        try:
            feature.attributes['HEARING_DATE'] = None
            dev_tlayer.edit_features(updates = dev_invalid_hearing_date)
        except Exception as e:
            print(e)

### 4. Update Features in Hosted Feature Layer

In [ ]:
if dev_update == True:
    from arcgis.features import GeoAccessor

In [ ]:
# Obtain Spatially Enabled DataFrame (SeDF) of the development applications
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.query
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.GeoAccessor.from_xy
if dev_update == True:
    dev = dev_tlayer.query()
    dev_df = dev.sdf
    dev_x = "X"
    dev_y = "Y"
    dev_wkid = 7991
    dev_sdf = GeoAccessor.from_xy(df = dev_df, x_column = dev_x, y_column = dev_y, sr = dev_wkid)

In [ ]:
# Update features using the Spatially Enabled DataFrame
# Slicing is necessary as the server may not be able to handle large dataset
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.GeoAccessor.to_featureset
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.delete_features
# Reference: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.FeatureLayer.edit_features
if dev_update == True:
    dev_item = gis.content.get(dev_itemid)
    dev_flayer = dev_item.layers[0]
    dev_flayer.delete_features(where = "F_ID is not null")
    i = 0
    batch_size = 250
    while i <= len(dev_sdf):
        try:
            dev_fs = dev_sdf.iloc[i:i + batch_size].spatial.to_featureset()
            dev_flayer.edit_features(adds = dev_fs)
            i += batch_size
        except:
            try:
                dev_fs = dev_sdf.iloc[i:i + 1].spatial.to_featureset()
                dev_flayer.edit_features(adds = dev_fs)
                i += 1
            except Exception as e:
                print(e)
                print(f"Application {dev_sdf.iloc[i].APPLICATION_NUMBER} cannot not be visualized.")
                i += 1